In [115]:
import pandas as pd
import numpy as np
import copy
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error,mean_absolute_error,r2_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [65]:
df=pd.read_csv("TrainAndValid.csv",low_memory=False)
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
(df["state"].dropna()).shape[0]

412698

In [68]:
print(df["MachineHoursCurrentMeter"].isnull().sum())
print(df["fiModelDesc"].isnull().sum())
print(df["fiBaseModel"].isnull().sum())
print(df["ProductSize"].isnull().sum())

265194
0
0
216605


In [69]:
df.columns.size

53

In [70]:
df.sort_values(by="saledate",inplace=True)
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
405675,4449186,46173.2,2318824,26964,173,99.0,1996,0.0,NaN,1/1/2012 0:00,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,NaN,NaN
268962,1806686,8000.0,1077085,9549,132,2.0,1998,NaN,NaN,1/10/2003 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221519,1691274,20500.0,1233251,22933,132,2.0,1994,NaN,NaN,1/10/2003 0:00,...,20 inch,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
283617,1895558,9000.0,1458959,6054,132,2.0,1994,NaN,NaN,1/10/2003 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
122970,1467430,17500.0,1339914,4200,132,2.0,1988,NaN,NaN,1/10/2003 0:00,...,20 inch,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [71]:
df[df["SalesID"]==1646770]["saledate"]

,saledate
205615,1/17/1989 0:00


In [72]:
df_copy=copy.deepcopy(df)

In [73]:
df_copy[df_copy["MachineID"]==1194089]["YearMade"]

,YearMade
274835,1980


In [74]:
df_copy['saledate'] = pd.to_datetime(df_copy['saledate'])
df_copy['saleYear'] = df_copy['saledate'].dt.year
df_copy['saleMonth'] = df_copy['saledate'].dt.month
df_copy['saleDay'] = df_copy['saledate'].dt.day
df_copy['saleDayOfWeek'] = df_copy['saledate'].dt.dayofweek
df_copy['saleDayOfYear'] = df_copy['saledate'].dt.dayofyear

In [75]:
print(df_copy["saleDayOfWeek"].isnull().sum())
print(df_copy["SalePrice"].isnull().sum())
print(df_copy["auctioneerID"].isnull().sum())
print(df_copy["MachineHoursCurrentMeter"].isnull().sum())

0
0
20136
265194


In [76]:
df_copy["auctioneerID"].isnull().sum()

20136

In [77]:
pd.Categorical(df_copy["state"]).codes

array([21, 37, 37, ..., 42, 43,  4], dtype=int8)

In [78]:
pd.Categorical(df_copy["UsageBand"]).codes

array([-1, -1, -1, ...,  1,  0,  2], dtype=int8)

In [79]:
for col in df.select_dtypes(include=['object']).columns:
  df_copy[col]=pd.Categorical(df_copy[col]).codes

In [80]:
df_copy.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleDay,saleDayOfWeek,saleDayOfYear
405675,4449186,46173.2,2318824,26964,173,99.0,1996,0.0,-1,3919,...,0,4,5,-1,-1,2012,1,1,6,1
268962,1806686,8000.0,1077085,9549,132,2.0,1998,NaN,-1,1932,...,-1,-1,-1,-1,-1,2003,1,10,4,10
221519,1691274,20500.0,1233251,22933,132,2.0,1994,NaN,-1,1932,...,-1,-1,-1,-1,-1,2003,1,10,4,10
283617,1895558,9000.0,1458959,6054,132,2.0,1994,NaN,-1,1932,...,-1,-1,-1,-1,-1,2003,1,10,4,10
122970,1467430,17500.0,1339914,4200,132,2.0,1988,NaN,-1,1932,...,-1,-1,-1,-1,-1,2003,1,10,4,10


In [81]:
df_copy = df_copy.fillna(-1)

In [82]:
for col in df_copy.columns:
    df_copy[col + "_is_missing"] = (df_copy[col] == -1).astype(bool)

In [63]:
df_copy[df_copy["SalesID"]==1646770]["Differential_Type_is_missing"]

,Differential_Type_is_missing
205615,True


In [84]:
df_copy["Differential_Type_is_missing"].isnull().sum()

0

In [98]:
train_set=df_copy[df_copy["saleYear"]!=2011]
validation_set=df_copy[df_copy["saleYear"]==2011]
train_set.shape,validation_set.shape

((377501, 116), (35197, 116))

In [101]:
X_train=train_set.drop(columns=["SalePrice"])
y_train=train_set["SalePrice"]
X_val=validation_set.drop(columns=["SalePrice"])
y_val=validation_set["SalePrice"]
X_train.shape,y_train.shape,X_val.shape,y_val.shape

((377501, 115), (377501,), (35197, 115), (35197,))

In [103]:
model=RandomForestRegressor(n_jobs=-1,
                            random_state=42,
                            max_samples=10000)

In [104]:
model.fit(X_train,y_train)

RandomForestRegressor(max_samples=10000, n_jobs=-1, random_state=42)

In [108]:
train_pred=model.predict(X_train)

In [110]:
def eval(y_true,y_pred):
  rmsle=np.sqrt(mean_squared_log_error(y_true,y_pred))
  mae=mean_absolute_error(y_true,y_pred)
  r2=r2_score(y_true,y_pred)
  return rmsle,mae,r2

In [111]:
eval(y_train,train_pred)

(0.25452492452124886, 5436.423592318961, 0.86491862038262)

In [112]:
val_pred=model.predict(X_val)

In [113]:
eval(y_val,val_pred)

(0.27890700030128396, 6639.903908855868, 0.8311642888953945)

In [114]:
rf_grid={"n_estimators": [30, 40, 50],
       "max_depth": [None, 3, 5, 10],
       "min_samples_split": [10, 14, 18],
       "min_samples_leaf": [1, 2, 3],
       "max_features": [0.5, 1, "sqrt", "auto"],
       "max_samples": [10000]}

In [118]:
random=RandomizedSearchCV(estimator=model,
                          param_distributions=rf_grid,
                          cv=5,
                          verbose=True,
                          n_iter=10,
                          n_jobs=-1,
                          random_state=42)
random.fit(X_train,y_train)
best_model_rcv=random.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [119]:
rcv_pred=best_model_rcv.predict(X_train)

In [120]:
eval(y_train,rcv_pred)

(0.3355173233877252, 7495.43079280343, 0.7627910787592848)

In [129]:
param_grid={"n_estimators": [170, 200],
            "min_samples_split": [4, 5],
            "min_samples_leaf": [2],
            "max_features": [0.4, 0.45, 0.5],
            "max_samples": [10000],
            "random_state":[42]}

In [131]:
grid=GridSearchCV(estimator=model,
                  cv=5,
                  param_grid=param_grid,
                  verbose=True,
                  n_jobs=-1)
grid.fit(X_train,y_train)
best_model_gcv=grid.best_estimator_

In [ ]:
gcv_pred=best_model_gcv.predict(X_train)

In [ ]:
eval(y_train,gcv_pred)

In [135]:
df_test=pd.read_csv("Test.csv",low_memory=False,parse_dates=["saledate"])
df_test.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1227829,1006309,3168,121,3,1999,3688.0,Low,2012-05-03,580G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227844,1022817,7271,121,3,1000,28555.0,High,2012-05-10,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1227847,1031560,22805,121,3,2004,6038.0,Medium,2012-05-10,EC210BLC,...,None or Unspecified,"9' 6""",Manual,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1227848,56204,1269,121,3,2006,8940.0,High,2012-05-10,330CL,...,None or Unspecified,None or Unspecified,Manual,Yes,Triple,NaN,NaN,NaN,NaN,NaN
4,1227863,1053887,22312,121,3,2005,2286.0,Low,2012-05-10,650K,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [136]:
df_test["Hydraulics"].isnull().sum()

2142